In [1]:
import pandas as pd
import numpy as np
import html
import nltk
import regex as re
import string
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
data = pd.read_csv('C:/Users/Danilo/Desktop/Documentos Acadêmicos/TERA/TERAprojetos/Projeto final/BNCC/repo/bncc-classifier/data/curated/data_eda_treat_will.csv')

In [3]:
data_clone = data.copy()

In [4]:
# renomeando as colunas do dataset

data_clone.columns = ['id', 'questoes', 'tipoQuestoes', 'topico', 'slug', 'materia', 'slug.1','etapaEnsino']
data_clone.drop(['slug', 'slug.1'], axis = 1, inplace = True)
data_clone.head()

,id,questoes,tipoQuestoes,topico,materia,etapaEnsino
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,Arte,Fundamental II
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,Matemática,Fundamental II
2,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,Química,Fundamental II
3,2016466,<p><strong>Anexe nesse espa&ccedil;o uma foto ...,4,Proposta,Língua Portuguesa,Fundamental I
4,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,4,Idade Moderna: Revolução Francesa: Antecedente...,História,Fundamental II


## Pre-processamento

* Escrevendo funcões para etapa de pre-processamento de Texto usando regex 

In [5]:
# Função para remover tags de html usando regex

def remove_tags(text: str) -> str:
    pattern = re.compile("<.*?>")
    cleantext = re.sub(pattern, " ", text).replace(u"\xa0", u" ")
    return cleantext

# Função para deixar todo o texto em caixa baixa
def to_lower(text: str) -> str:
    return text.lower()

# Função para remover numeros
def remove_numbers(text: str) -> str:
    pattern = re.compile("[0-9]+")
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover stopwords em portugues; preposições, conjunções, artigos, etc
def remove_standard_stopwords_pt(text: str) -> str:
    stop_words = set(stopwords.words("portuguese"))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    final_sentence = " ".join(filtered_sentence)
    return final_sentence  

def remove_standard_stopwords_en(text: str) -> str:
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    final_sentence = " ".join(filtered_sentence)
    return final_sentence  

# Função para remover caracteres especiais e pontuação
def remove_punctuation(text: str) -> str:
    pattern = re.compile("[\d\.\,\*):(\?\/\-\]\[\–º_]")
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas em itálico
def remove_italic_dquotes(text: str) -> str:
    pattern = re.compile(r'"')
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas de abertura
def remove_open_dquotes(text: str) -> str:
    pattern = re.compile(r'“')
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas de abertura
def remove_end_dquotes(text: str) -> str:
    pattern = re.compile(r'”')
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas de abertura
def remove_italic_quotes(text: str) -> str:
    pattern = re.compile(r"'")
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas de abertura
def remove_open_quotes(text: str) -> str:
    pattern = re.compile(r'‘')
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas de abertura
def remove_end_quotes(text: str) -> str:
    pattern = re.compile(r'’')
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas de abertura
def remove_quote(text: str) -> str:
    pattern = re.compile(r'‛')
    clean_text = re.sub(pattern, " ", text)
    return clean_text

In [6]:
# resolve problema de codificação html da coluna 'questoes'
data_clone['questoes_html']= data_clone['questoes'].astype('str').apply(html.unescape)

# limpando o texto utilizando as funçoes criadas
data_clone['questoes_tags'] = data_clone['questoes_html'].apply(remove_tags)
data_clone['questoes_lower'] = data_clone['questoes_tags'].apply(to_lower)
data_clone['questoes_numbers'] = data_clone['questoes_lower'].apply(remove_numbers)

### limpando pontuações e todos os tipos de aspas

data_clone['questoes_punct'] = data_clone['questoes_numbers'].apply(remove_punctuation)

data_clone['questoes_no_italic_quotes'] = data_clone['questoes_punct'].apply(remove_italic_quotes)
data_clone['questoes_no_open_quotes'] = data_clone['questoes_no_italic_quotes'].apply(remove_open_quotes)
data_clone['questoes_no_end_quotes'] = data_clone['questoes_no_open_quotes'].apply(remove_end_quotes)

data_clone['questoes_no_italic_dquotes'] = data_clone['questoes_no_end_quotes'].apply(remove_italic_dquotes)
data_clone['questoes_no_open_dquotes'] = data_clone['questoes_no_italic_dquotes'].apply(remove_open_dquotes)
data_clone['questoes_no_end_dquotes'] = data_clone['questoes_no_open_dquotes'].apply(remove_end_dquotes)

data_clone['questoes_no_quotes'] = data_clone['questoes_no_end_dquotes'].apply(remove_quote)

In [ ]:
data_clone['questoes_no_quotes'][0]

In [8]:
nlp = spacy.load('pt_core_news_sm')

In [9]:
docs = list(nlp.pipe(data_clone.questoes_no_quotes))

In [10]:
def extract_tokens_plus_meta(doc:spacy.tokens.doc.Doc):
    """Extract tokens and metadata from individual spaCy doc."""
    return [
        (i.text, i.i, i.lemma_, i.ent_type_, i.tag_, 
         i.dep_, i.pos_, i.is_stop, i.is_alpha, 
         i.is_digit, i.is_punct) for i in doc
    ]

In [14]:
def tidy_tokens(docs):
    """Extract tokens and metadata from list of spaCy docs."""
    
    cols = [
        "doc_id", "token", "token_order", "lemma", 
        "ent_type", "tag", "dep", "pos", "is_stop", 
        "is_alpha", "is_digit", "is_punct"
    ]
    
    meta_df = []
    for ix, doc in enumerate(docs):
        meta = extract_tokens_plus_meta(doc)
        meta = pd.DataFrame(meta)
        meta.columns = cols[1:]
        meta = meta.assign(doc_id = ix).loc[:, cols]
        meta_df.append(meta)
        
    return pd.concat(meta_df) 

In [18]:
tidy_tokens(docs)

,doc_id,token,token_order,lemma,ent_type,tag,dep,pos,is_stop,is_alpha,is_digit,is_punct
0,0,,0,,,SPACE,dep,SPACE,False,False,False,False
1,0,com,1,com,,ADP,case,ADP,True,True,False,False
2,0,base,2,base,,NOUN,obl,NOUN,False,True,False,False
3,0,em,3,em,,ADP,case,ADP,True,True,False,False
4,0,seus,4,seu,,DET,det,DET,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
23,83674,vendeu,23,vender,,VERB,parataxis,VERB,False,True,False,False
24,83674,nesses,24,nesses,,ADJ,amod,ADJ,False,True,False,False
25,83674,três,25,três,,NUM,nummod,NUM,True,True,False,False
26,83674,dias,26,dia,,NOUN,obj,NOUN,False,True,False,False
